## Imports

In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash
from dash.dependencies import Input, Output

import numpy as np
from sympy import *
import pandas as pd

## Settings variables and solving for Pi's

In [2]:
a,m,si,E,L2,L1,L = symbols("W_a,W_m,s_i,E_m,L_{II},L_I,L")
pi_eq = symbols(pretty(pi)+"_eq")
pi_m = symbols(pretty(pi)+"_M")
s_eq = symbols("s\u0302")

In [3]:
Pi_eq = a/((a*si)+(1-si)*m)
Pi_eq

W_a/(W_a*s_i + W_m*(1 - s_i))

In [4]:
Pi_M = E/(L1 + si*L2)
Pi_M

E_m/(L_I + L_{II}*s_i)

In [5]:
intersection = Eq(Pi_eq,Pi_M)

In [6]:
Eq(s_eq,solve(intersection,si)[0])

Eq(ŝ, (E_m*W_m - L_I*W_a)/(-E_m*W_a + E_m*W_m + L_{II}*W_a))

#### Solving for numerical intersection for graph

In [7]:
s_eq = solve(intersection,si)[0]
param_dict = {"W_m":12,
              "W_a":2,
              "E_m":50,
              "L_I":80,
              "L_{II}":120}
s_eq.subs(param_dict)

22/37

In [8]:
Pi_M.subs({"E_m":50,
           "L_I":80,
           "L_{II}":120,
           "s_i":22/37})

0.330357142857143

In [9]:
s_eq_func = lambdify([m,a,E,L,L1],s_eq.subs(L2,(L-L1)),"numpy")

#### Dashing UP

In [10]:
Pi_eq_func = lambdify([a,m,si],Pi_eq,"numpy")
Pi_M_func = lambdify([E,L1,L2,si],Pi_M,"numpy")

In [11]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Plotting Pi's"),
    dcc.Graph(id='graph'),
    html.Div([
        html.Div(["Wm",
        dcc.Input(
            id='m_input',
            value=8, type="number")
            ]),
        html.Br(),
        html.Div(["Wa:",
            dcc.Input(
                id="a_input",
                value=4, type="number")
            ]),
        html.Br(),
        html.Div(["E:",
            dcc.Input(
                id="E_input",
                value=20, type="number")
            ]),
        html.Br(),
        html.Div(["L:",
            dcc.Input(
                id="L_input",
                value=100, type="number")
            ]),
        html.Br(),
        html.Div(["L1:",
            dcc.Input(
                id="L1_input",
                value=30, type="number")
            ])])
],style={"width": "60%","padding-left":"20%", "padding-right":"20%"})
    

@app.callback(
    Output("graph","figure"),
    [Input("m_input","value"),
    Input("a_input","value"),
    Input("E_input","value"),
    Input("L_input","value"),
    Input("L1_input","value")])
def update_graph(m,a,E,L,L1):
    s = np.linspace(0,1,1000)
    y1 = Pi_eq_func(a,m,s)
    y2 = Pi_M_func(E,L,L1,s)
    df = pd.DataFrame({"s_i":s,"Pi_eq":y1,"Pi_M":y2})
    fig = px.line(df,y=["Pi_eq","Pi_M"],x="s_i")
    return fig



In [12]:
def start_app(port=8060):
    if __name__ == '__main__':
        app.run_server(debug=True,port=port)

In [13]:
def stop_app(port=8060):
    app._terminate_server_for_port("localhost",port)

## Solving for $L_I$

#### Uniform distribution

We know from the Uniform ~ U(0,1) that

CDF: $F(s_i < t) = t$

Hence, from 
$$F(s_i < \bar{s}) = \bar{s}$$
we get the following (quadratic!) Equation:

In [14]:
endog = Eq(((a*L1-E*m)/(E*(a-m)-a*(L-L1)))*L,L1)
endog

Eq(L*(-E_m*W_m + L_I*W_a)/(E_m*(W_a - W_m) - W_a*(L - L_I)), L_I)

In [15]:
len(solve(endog,L1))

2

In [16]:
solve(endog,L1)[0]

(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)

In [17]:
solve(endog,L1)[1]

(-E_m*W_a + E_m*W_m + 2*L*W_a + sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)

In [18]:
L1_sol0 = lambdify([m,a,E,L],solve(endog,L1)[0],"numpy")

In [19]:
L1_sol1 = lambdify([m,a,E,L],solve(endog,L1)[1],"numpy")

In [20]:
print(L1_sol0(8,2,40,100))
print(L1_sol1(8,2,40,100))

62.02041028867288
257.9795897113271


In [21]:
print(L1_sol0(8,3,300,500))
print(L1_sol1(8,3,300,500))

346.88711258507254
1153.1128874149274


If any, its L1_sol0.

In [22]:
L1_sol = symbols("L1^*")
Eq(L1_sol,simplify(solve(endog,L1)[0]))

Eq(L1^*, (-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a))

In [23]:
L1_sol = solve(endog,L1)[0]

## Solve for $L_m$ and U

The major question in solving for the ex-post allocations is the "true" search parameter, or rather how it is related to the distribution of beliefs. Here, the entire weight is on choosing a normatively appealing relation. The most simple one, would be, of course, that of rational expectations. Instead of for the individual, this can at most hold for the average: $$\bar{s} = \frac{\sum_{i}{s_i}}{L}$$

Solving for $$L_m = L_I + L_{II}\bar{s}\pi = L_I + (L-L_{I})\bar{s}\pi = L_I(1-\pi\bar{s})+ L\pi\bar{s}$$

In [24]:
Pi_M

E_m/(L_I + L_{II}*s_i)

In [25]:
L_m, U, s_t = symbols("L_m,U,sbar")
# Modifying Pi_m to represent the TRUE market hiring rate, not the believed one
Pi_m = E/(L1*(1-s_t)+L*s_t)
Pi_m

E_m/(L*sbar + L_I*(1 - sbar))

In [26]:
L_m = L1*(1-Pi_m*s_t) + L*Pi_m*s_t
L_m

E_m*L*sbar/(L*sbar + L_I*(1 - sbar)) + L_I*(-E_m*sbar/(L*sbar + L_I*(1 - sbar)) + 1)

In [27]:
L_m_sol = L_m.subs(L1,L1_sol)
L_m_sol

E_m*L*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + (-E_m*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + 1)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)

In [28]:
L_m_sol_func = lambdify([m,a,E,L,s_t],L_m_sol,"numpy")

In [29]:
U = L_m - E
U

E_m*L*sbar/(L*sbar + L_I*(1 - sbar)) - E_m + L_I*(-E_m*sbar/(L*sbar + L_I*(1 - sbar)) + 1)

In [30]:
U_sol = U.subs(L1,L1_sol)
U_sol

E_m*L*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) - E_m + (-E_m*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + 1)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)

In [31]:
Unemployment_rate = U_sol/L_m_sol
Unemployment_rate

(E_m*L*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) - E_m + (-E_m*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + 1)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a))/(E_m*L*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + (-E_m*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + 1)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a))

In [32]:
# Alternatively
UR2 = -E/L_m_sol
UR2

-E_m/(E_m*L*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + (-E_m*sbar/(L*sbar + (1 - sbar)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + 1)*(-E_m*W_a + E_m*W_m + 2*L*W_a - sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a))

In [33]:
unemployment_rate_func = lambdify([m,a,E,L,s_t],Unemployment_rate,"numpy")

### Getting the First Best Solutions and they lambdified functions

In [34]:
L_m_FB = L+ E*(m/a) + (m*L)/((m-a)*s_t-m)
L_m_FB

E_m*W_m/W_a + L*W_m/(-W_m + sbar*(-W_a + W_m)) + L

In [35]:
L_m_FB_func = lambdify([m,a,E,L,s_t],L_m_FB,"numpy")

In [36]:
UR_FB  = (L_m_FB - E)/L_m_FB
UR_FB_func = lambdify([m,a,E,L,s_t],UR_FB,"numpy")

### Plotting my solutions against the FB

In [37]:
import dash_bootstrap_components as dbc

app = JupyterDash(__name__,external_stylesheets=[dbc.themes.GRID])

#Todo put graphs beside each other!


app.layout = html.Div([
    html.Div([
        dbc.Row([
            dbc.Col(dcc.Graph(id='graph'),align="start"),
            dbc.Col(dcc.Graph(id="graph2"),align="center"),
            dbc.Col(dcc.Graph(id="graph3"),align="end")
                ])
            ]),
    
    html.Div([
        dbc.Row([
            dbc.Col(
                html.Div(["Wm",
                    dcc.Input(
                        id='m_input',
                        value=12, type="number")
                        ])
                    ),
            dbc.Col(
                html.Div(["Wa:",
                    dcc.Input(
                        id="a_input",
                        value=4, type="number")
                        ])
                    ),
            dbc.Col(
                html.Div(["E:",
                    dcc.Input(
                        id="E_input",
                        value=50, type="number")
                        ])
                    )
                ]),
        
        dbc.Row([
            dbc.Col(
                html.Div(["L:",
                    dcc.Input(
                        id="L_input",
                        value=100, type="number")
                        ])
                    ),
            dbc.Col(
                html.Div(["S_true:",
                    dcc.Slider(
                        id="s_true_input", min=0, max=1, step=0.1,
                        value=0.5)     
                        ])
                    )
                ])
            ]),
])
    

@app.callback(
    Output("graph","figure"),
    [Input("m_input","value"),
    Input("a_input","value"),
    Input("E_input","value"),
    Input("L_input","value")])
def update_graph(m,a,E,L):
    s = np.linspace(0,1,1000)
    L1 = L1_sol0(m,a,E,L)
    y1 = Pi_eq_func(a,m,s)
    y2 = Pi_M_func(E,L1,L-L1,s)
    df = pd.DataFrame({"s_i":s,"Pi_eq":y1,"Pi_M":y2})
    fig = px.line(df,y=["Pi_eq","Pi_M"],x="s_i")
    fig.add_vline(x=s_eq_func(m,a,E,L,L1))
    return fig

@app.callback(
    Output("graph2","figure"),
    [Input("m_input","value"),
    Input("a_input","value"),
    Input("E_input","value"),
    Input("L_input","value"),
    Input("s_true_input","value")])
def update_ex_post_graph(m,a,E,L,s):
    bar = pd.DataFrame({"Title":["L_I","L_m","L_m_Fields"], "Solution": [L1_sol0(m,a,E,L),L_m_sol_func(m,a,E,L,s),L_m_FB_func(m,a,E,L,s)]})
    fig = px.bar(bar,x="Title",y="Solution")
    return fig

@app.callback(
    Output("graph3","figure"),
    [Input("m_input","value"),
    Input("a_input","value"),
    Input("E_input","value"),
    Input("L_input","value"),
    Input("s_true_input","value")])
def update_ex_post_graph(m,a,E,L,s):
    bar = pd.DataFrame({"Title":["Unemployment Rate","Unemployment Rate Fields"], "Solution": [unemployment_rate_func(m,a,E,L,s),UR_FB_func(m,a,E,L,s)]})
    fig = px.bar(bar,x="Title",y="Solution")
    return fig
    


In [38]:
start_app(8080)

Dash app running on http://127.0.0.1:8080/


In [39]:
stop_app(8080)

From the visualisations one can see that unemployment rates tend to be much lower compared to the solution in Fields. 
\begin{itemize}
    \item why is that so mathematically?
    \item Why is that so intuitively?
\end{itemize}    


In [43]:
UR_diff = UR_FB - UR2
simplify(UR_diff)

E_m/(E_m*L*sbar/(L*sbar + (sbar - 1)*(E_m*W_a - E_m*W_m - 2*L*W_a + sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) + (E_m*sbar/(L*sbar + (sbar - 1)*(E_m*W_a - E_m*W_m - 2*L*W_a + sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) - 1)*(E_m*W_a - E_m*W_m - 2*L*W_a + sqrt(E_m**2*W_a**2 - 2*E_m**2*W_a*W_m + E_m**2*W_m**2 - 4*E_m*L*W_a**2 + 4*L**2*W_a**2))/(2*W_a)) - (E_m - E_m*W_m/W_a + L*W_m/(W_m + sbar*(W_a - W_m)) - L)/(E_m*W_m/W_a - L*W_m/(W_m + sbar*(W_a - W_m)) + L)

In [45]:
# With L1
UR_diff_pre_sol = UR_FB - (U/L_m)
UR_diff_pre_sol

(-E_m + E_m*W_m/W_a + L*W_m/(-W_m + sbar*(-W_a + W_m)) + L)/(E_m*W_m/W_a + L*W_m/(-W_m + sbar*(-W_a + W_m)) + L) - (E_m*L*sbar/(L*sbar + L_I*(1 - sbar)) - E_m + L_I*(-E_m*sbar/(L*sbar + L_I*(1 - sbar)) + 1))/(E_m*L*sbar/(L*sbar + L_I*(1 - sbar)) + L_I*(-E_m*sbar/(L*sbar + L_I*(1 - sbar)) + 1))